## Several ways to conduct factor preprocessing in FROZEN

### 1. Use a decorator `@prep`

In [1]:
from frozen.engine import FrozenBt
from frozen.factor import *
from frozen.utils import Universe

class FactorFactory(FrozenBt):

    def __init__(self):
        super().__init__(__vsc_ipynb_file__)
    
    def univ(self):
        universe = Universe(self.config)
        self.universe = universe.pool
    
    def prepare_data(self):
        data_definitions = [
            ('stock_daily_hfq', ('close', 'pct_chg'), ('close', 'returns')),
        ]
        return self.dataloader.load_batch(data_definitions, self.universe, start_date=self.start_date_lookback, end_date=self.end_date)
    
    @prep(method="standardize", cross_section=True)
    def calc(self):
        string = "rank(Ts_ArgMax(SignedPower(where(returns < 0 ? stddev(returns, 5) : close), 2.0), 5))"
        alpha = calc_str(string, self.prepare_data())
        return alpha

c:\Users\zhaoxs3\AppData\Local\miniforge3\envs\frozen\Lib\site-packages\tqdm_joblib\__init__.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### 2. Wrap factor directly in `calc`

In [2]:
from frozen.engine import FrozenBt
from frozen.factor import *
from frozen.utils import Universe

class FactorFactory(FrozenBt):

    def __init__(self):
        super().__init__(__vsc_ipynb_file__)
    
    def univ(self):
        universe = Universe(self.config)
        self.universe = universe.pool
    
    def prepare_data(self):
        data_definitions = [
            ('stock_daily_hfq', ('close', 'pct_chg'), ('close', 'returns')),
        ]
        return self.dataloader.load_batch(data_definitions, self.universe, start_date=self.start_date_lookback, end_date=self.end_date)
    
    def calc(self):
        string = "rank(Ts_ArgMax(SignedPower(where(returns < 0 ? stddev(returns, 5) : close), 2.0), 5))"
        alpha = calc_str(string, self.prepare_data())
        alpha = clip_mad(normalize(alpha, cross_section=False, expanding=True), expanding=False, window=5)
        return alpha

### 3. Use chaining/pipelining operations

In [3]:
from frozen.engine import FrozenBt
from frozen.factor import *
from frozen.utils import Universe

class FactorFactory(FrozenBt):

    def __init__(self):
        super().__init__(__vsc_ipynb_file__)
    
    def univ(self):
        universe = Universe(self.config)
        self.universe = universe.pool
    
    def prepare_data(self):
        data_definitions = [
            ('stock_daily_hfq', ('close', 'pct_chg'), ('close', 'returns')),
        ]
        return self.dataloader.load_batch(data_definitions, self.universe, start_date=self.start_date_lookback, end_date=self.end_date)
    
    def calc(self):
        string = "rank(Ts_ArgMax(SignedPower(where(returns < 0 ? stddev(returns, 5) : close), 2.0), 5))"
        alpha = calc_str(string, self.prepare_data())
        alpha = alpha.normalize(cross_section=True).winsorize()
        return alpha